In [1]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.colors as mcolors

import numpy as np
import pandas as pd
import pickle

from scipy.signal import savgol_filter  
from scipy.ndimage import median_filter

import os, sys

# sys.path.append('/Users/mayem1/Documents/Code/Eureka/src')
from eureka.lib import manageevent as me

import astraeus.xarrayIO as xrio

## USER INPUTS ##
fin_tdir = '/Users/ashtar1/'
S4_v1 = 'DataAnalysis/HST/HD86226c/Optimized/Visit04-07/Optimized_2024-03-20_HD86226c_run1/White/S4/'
S5_v1 = 'DataAnalysis/HST/HD86226c/Optimized/Visit04-07/Optimized_2024-03-20_HD86226c_run1/White/S5/'

S4_savefile = 'S4_HD86226c_Meta_Save'
S5_savefile = 'S5_HD86226c_Meta_Save'
h5 = 'S4_HD86226c_ap79_bg80_LCData.h5'

# meta_v1 = me.loadevent(fin_tdir+S4_v1+S4_savefile)
meta_v1 = me.loadevent(fin_tdir+S5_v1+S5_savefile)

t0 = 60324.31034 


## Calculations ##
cm = np.flip(np.loadtxt('/Users/ashtar1/Documents/ScientificColourMaps8/roma/roma.txt'), axis = 0)
mymap = mcolors.LinearSegmentedColormap.from_list('my_colormap', cm)

norm=matplotlib.colors.Normalize(vmin=2.75,vmax=5.25)
colors=mymap
scal_m=matplotlib.cm.ScalarMappable(cmap=colors,norm=norm)
scal_m.set_array([])

wave_v1     = meta_v1.wave
wave_low_v1 = meta_v1.wave_low
wave_hi_v1  = meta_v1.wave_hi

nints_v1 = xrio.readXR(fin_tdir+S4_v1+h5).time.values.shape[0]
nints_v1 = len(meta_v1.time)
# print(nints_v1)

### Read in LCS

wave = wave_v1
nints = nints_v1
print(meta_v1)

data  = np.empty([len(wave),nints])*np.nan
err   = np.empty([len(wave),nints])*np.nan
model = np.empty([len(wave),nints])*np.nan
resid = np.empty([len(wave),nints])*np.nan

# print(data.shape)
norm = 1

counter = 0
for filef in sorted(os.listdir(fin_tdir+S5_v1)):
    if filef.endswith('.txt'):
        wave_ind = int(filef.split('_')[-1].split('.')[0].split('ch')[-1])
        
        openh = pd.read_csv(fin_tdir+S5_v1+filef, sep=" ", comment='#')
        # openh = pd.read_csv(fin_tdir+S5_v1+filef, sep=" ", header=13)

        print(openh.columns)
        print(openh)
        
        data[wave_ind,:]  = openh['lcdata']/np.nanmean(openh['lcdata'][:norm])
        err[wave_ind,:]   = openh['lcerr']/np.nanmean(openh['lcdata'][:norm])
        model[wave_ind,:] = openh['model']/np.nanmean(openh['lcdata'][:norm])
        resid[wave_ind,:] = data[wave_ind,:] - model[wave_ind,:]
        counter +=1

time = (xrio.readXR(fin_tdir+S4_v1+h5).time.values - t0)*24

ms = 4
alp = 0.8

y_major = 0.25
y_minor = 0.05

xmin = np.nanmin(time)-0.02*24
xmax = np.nanmax(time)+0.02*24

ymin = 0.9
ymax = 3.5

x_major = 1
x_minor = 0.25

#### VISIT AND REDUCTION COMPARISON PLOT

figsize = ((8.5-2.0)*4.0,2.5*(11.-2.))

fig,ax = plt.subplots(1,4, figsize = figsize)
fig.dpi = 300

plt.subplots_adjust(hspace = 0.0, wspace = 0.0)

ms = 15
alp = 0.6

norm=matplotlib.colors.Normalize(vmin=0,vmax=len(wave_v1))
colors=mymap
scal_m=matplotlib.cm.ScalarMappable(cmap=colors,norm=norm)
scal_m.set_array([])


offset = 0
offfac = 0.1
for bini in range(nints_v1):

    ax[0].plot(time, data[bini,:] + offset,'.', ms = ms, color = 'white', alpha = 1.0, zorder = 100)
    ax[0].plot(time, data[bini,:] + offset ,'.', ms = ms, color = scal_m.to_rgba(bini), alpha = alp, zorder = 100)
    ax[0].plot(time, model[bini,:] + offset, '-', lw = 0.5, color = 'black', zorder = 105)
    
    if bini == 0:
        offset += 1.2*offfac
    else:
        offset += offfac
    
offset = 1
offfac = 0.1
for bini in range(nints_v1):
   
    ax[1].plot(time, resid[bini,:] + offset,'.', ms = ms, color = 'white', alpha = 1.0, zorder = 100)
    ax[1].plot(time, resid[bini,:] + offset ,'.', ms = ms, color = scal_m.to_rgba(bini), alpha = alp, zorder = 100)
    ax[1].hlines(y = offset, xmin = time[0], xmax = time[-1], lw = 0.5, color = 'black', zorder = 105)
    
    if bini == 0:
        offset += 1.2*offfac
    else:
        offset += offfac
        
        
#formatting
for a in ax:
    a.set_facecolor('gainsboro')
    a.yaxis.set_major_locator(MultipleLocator(y_major))
    a.yaxis.set_minor_locator(MultipleLocator(y_minor))

    
    a.set_ylim(ymin,ymax) 
    
    a.tick_params(axis = 'both', which = 'major', direction = 'inout', top = 'True', right = 'True',
        labelsize = 25, length = 6, width = 2.0)
    a.tick_params(axis = 'both', which = 'minor', direction = 'in', top = 'True', right = 'True',
                labelsize = 25, length = 2, width = 0.7, color = 'black')

    a.xaxis.set_major_locator(MultipleLocator(x_major))
    a.xaxis.set_minor_locator(MultipleLocator(x_minor))

    a.set_xlim(xmin,xmax)

ax[1].set_yticklabels([])  
ax[2].set_yticklabels([])  
ax[3].set_yticklabels([]) 

ax[0].set_ylabel('Relative Flux + Offset', fontsize = 35)
plt.figtext(0.5,0.075, 'Time [hours]', ha = 'center', va = 'center', fontsize = 35)

plt.savefig('Users/ashtar1/Desktop/LCFits_all.png', bbox_inches = 'tight', dpi = fig.dpi, facecolor = 'white')

Finished loading parameters from /Users/ashtar1/DataAnalysis/HST/HD86226c/Optimized/Visit04-07/Optimized_2024-03-20_HD86226c_run1/White/S4/S4_HD86226c_ap79_bg80_LCData.h5
ncpu: 8
multwhite: False
multspec: False
allapers: False
rescale_err: False
fit_par: ./s5_fit_par_HD86226c.epf
verbose: True
fit_method: emcee
run_myfuncs: [batman_tr,polynomial,hstramp]
maskFirstOrbit: False
maskFirstFrames: False
paramsExoMAST: True
divide_white: False
residuals_white_dir: /Users/ashtar1/DataAnalysis/HST/HD86226c/Optimized/Visit28-31/Optimized_2024-03-09_HD86226c_run1/White/S5/S5_HD86226c_ap79_bg80_Table_Save_ch0.txt
manual_clip: [[ 0 26]
 [26 27]
 [52 53]
 [78 79]]
use_generate_ld: exotic-ld
ld_file: /Users/ashtar1/Data/exotic-ld_data/Sensitivity_files/HST_WFC3_G141_throughput.csv
ld_file_white: /Users/ashtar1/Data/exotic-ld_data/Sensitivity_files/HST_WFC3_G141_throughput.csv
old_fitparams: None
lsq_method: Powell
lsq_tol: 1e-08
lsq_maxiter: None
old_chain: None
lsq_first: True
run_nsteps: 1000
run

ValueError: x and y must have same first dimension, but have shapes (104,) and (75,)